<a href="https://colab.research.google.com/github/khorabi/python-hashtagtreinamentos/blob/main/projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio:

Você faz parte da equipe de Analytics de uma grande marca de vestuário com mais de 25 lojas espalhadas em Shoppings de todo o Brasil.

Toda semana você precisa enviar para a diretoria um ranking atualizado com as 25 lojas contendo 3 informações:
- Faturamento de cada Loja
- Quantidade de Produtos Vendidos de cada Loja
- Ticket Médio dos Produto de cada Loja

Além disso, cada loja tem 1 gerente que precisa receber o resumo das informações da loja dele. Por isso, cada gerente deve receber no e-mail:
- Faturamento da sua loja
- Quantidade de Produtos Vendidos da sua loja
- Ticket Médio dos Produto da sua Loja

Esse relatório é sempre enviado como um resumo de todos os dados disponíveis no ano.


# Solução do Desafio:

Para resolver o desafio vamos seguir a seguinte lógica:

- Passo 1 - Importar a base de Dados
- Passo 2 - Visualizar a Base de Dados para ver se precisamos fazer algum tratamento
- Passo 3 - Calcular os indicadores de todas as lojas:
  - Faturamento por Loja
  - Quantidade de Produtos Vendidos por Loja
  - Ticket Médio dos Produto por Loja
- Passo 4 - Calcular os indicadores de cada loja
- Passo 5 - Enviar e-mail para cada loja
- Passo 6 - Enviar e-mail para a diretoria

passo 1/2

In [12]:
import pandas as pd

tabela_vendas = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/python-hashtagtreinamentos/Vendas.xlsx")
#display(tabela_vendas)
tabela_vendas.head()

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600
4,2,2019-01-02,Norte Shopping,Chinelo Liso,1,71,71


passo 3

In [10]:
tabela_faturamento = tabela_vendas[['ID Loja', 'Valor Final']].groupby('ID Loja').sum()
tabela_faturamento = tabela_faturamento.sort_values(by="Valor Final", ascending=False)
display(tabela_faturamento)

,Valor Final
ID Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


In [15]:
tabela_quantidade = tabela_vendas[['ID Loja', 'Quantidade']].groupby('ID Loja').sum()
#tabela_quantidade = tabela_quantidade.sort_values(by="Quantidade", ascending=False)
display(tabela_quantidade)

,Quantidade
ID Loja,
Bourbon Shopping SP,8863
Center Shopping Uberlândia,8627
Iguatemi Campinas,8935
Iguatemi Esplanada,8580
Norte Shopping,9014
Novo Shopping Ribeirão Preto,8751
Palladium Shopping Curitiba,9091
Parque Dom Pedro Shopping,8388
Passei das Águas Shopping,8618


In [22]:
ticket_medio = (tabela_faturamento['Valor Final'] / tabela_quantidade['Quantidade']).to_frame()
ticket_medio = ticket_medio.rename(columns={0: "Ticket Medio"})
ticket_medio.head()

,Ticket Medio
ID Loja,
Bourbon Shopping SP,194.754598
Center Shopping Uberlândia,193.453228
Iguatemi Campinas,197.248909
Iguatemi Esplanada,198.098019
Norte Shopping,189.923231


função email

In [27]:
def enviar_email(nome_da_loja, tabela):
  import smtplib
  import email.message

  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = f"""
  <p>Prezados,</p>
  <p>Segue relatório de vendas</p>
  {tabela.to_html()}
  """ #editar
    
  msg = email.message.Message()
  msg['Subject'] = f"Relatório de vendas - {nome_da_loja}" #editar
    
  # Fazer antes (apenas na 1ª vez): Ativar Aplicativos não Seguros.
    # Gerenciar Conta Google -> Segurança -> Aplicativos não Seguros -> Habilitar
  # Caso mesmo assim dê o erro: smtplib.SMTPAuthenticationError: (534,
  # Você faz o login no seu e-mail e depois entra em: https://accounts.google.com/DisplayUnlockCaptcha
  msg['From'] = 'exemploemail@gmail.com' #editar
  msg['To'] = 'exemploemail@gmail.com' #editar
  password = "exemplosenha" #editar
  msg.add_header('Content-Type', 'text/html') 
  msg.set_payload(corpo_email )
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')

envio de email

In [30]:
tabela_completa = tabela_faturamento.join(tabela_quantidade).join(ticket_medio)
enviar_email("Diretoria", tabela_completa)

Email enviado


email para cada loja

In [31]:
lojas = tabela_vendas['ID Loja'].unique()

for loja in lojas:
  tabela_loja = tabela_vendas.loc[tabela_vendas['ID Loja'] == loja, ['ID Loja', 'Quantidade', 'Valor Final']]
  resumo_loja = tabela_loja.groupby('ID Loja').sum()
  resumo_loja['Ticket Médio'] = resumo_loja['Valor Final'] / resumo_loja['Quantidade']
  enviar_email(loja, resumo_loja)

Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
